In [1]:
from selenium import webdriver
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup
import re
import time
from datetime import date, timedelta

In [2]:
driver = webdriver.Chrome('/usr/local/bin/chromedriver')

In [3]:
root = 'https://rotogrinders.com'

In [4]:
def get_base_page(date, Fanduel=False):
    if Fanduel:
        url = "https://rotogrinders.com/resultsdb/site/fanduel/date/{}/sport/nba".format(date)
        return driver.get(url)
    else:
        url = "https://rotogrinders.com/resultsdb/site/draftkings/date/{}/sport/nba".format(date)
        return driver.get(url)
def click_ownership_page():
    for elt in driver.find_elements_by_class_name("ant-tabs-tab"):
        if "Ownership" in elt.text:
            elt.click()
            return
    raise Exception('no button')
def click_summary_page():
    for elt in driver.find_elements_by_class_name("ant-tabs-tab"):
        if "Summary" in elt.text:
            elt.click()
            return
    raise Exception('no button')
def click_contests_page():
    for elt in driver.find_elements_by_class_name("ant-tabs-tab"):
        if "Contests" in elt.text:
            elt.click()
            return
    raise Exception('no button')
def click_right():
    driver.find_elements_by_css_selector('.ant-btn.ant-btn-icon-only')[1].click()
def get_winning_results():
    soup=BeautifulSoup(driver.page_source,'html.parser')
    tables = soup.findAll("table")
    table=pd.read_html(str(tables[0]))[0]
    for i in range(200):
        try:
            click_right()
        except:
            break
        time.sleep(.02)
        soup=BeautifulSoup(driver.page_source,'html.parser')
        tables = soup.findAll("table")
        a =pd.read_html(str(tables[0]))[0]
        table = table.append(a)
    return table
def get_actual_points():
    time.sleep(3)
    click_ownership_page()
    time.sleep(1)
    soup=BeautifulSoup(driver.page_source,'html.parser')
    tables = soup.findAll("table")
    df = pd.read_html(str(tables[2]))[0]
    return df
def get_contests(date):
    output = []
    time.sleep(.1)
    try:
        click_contests_page()
    except:
        return output
    time.sleep(.1)
    soup=BeautifulSoup(driver.page_source,'html.parser')
    tables = soup.findAll("table")
    #print(tables)
    index = 0 if len(tables) == 1 else 2
    rows = tables[index].find("tbody").find_all("tr")
    records = pd.read_html(str(tables[index]))[0].to_dict(orient='records')
    for i in range(len(rows)):
        #print(rows[i].find_all('a', href=True))
        if float(records[i]['Top Prize'][1:].replace(",","")) > 4000 and float(records[i]['Buy In'][1:].replace(",","")) < 30 and float(records[i]['Max Entries']) >= 10: 
            print(root+rows[i].find_all('a', href=True)[0]['href'])
            driver.get(root+rows[i].find_all('a', href=True)[0]['href'])
            time.sleep(10)
            print(records[i])
            df = get_winning_results()
            records[i]['PAYOUTS']=df
            output.append(records[i])
    return output
def projections(date_string):
    output = {}
    url = 'https://rotogrinders.com/projected-stats/nba-player?site=draftkings&date={}'.format(date_string)
    driver.get(url)
    #print('hello')
    soup=BeautifulSoup(driver.page_source,'html.parser')
    mydivs = soup.findAll("div", {"class": "rgt-col"})
    try:
        floor = [float(a.text) for a in mydivs[-3].findAll("div")[1:]]
        ceil = [float(a.text) for a in mydivs[-4].findAll("div")[1:]]
        points = [float(a.text) for a in mydivs[-2].findAll("div")[1:]]
        name = [a.text for a in mydivs[0].findAll("div")[1:]]
        salary = [float(a.text[1:-1])*1000 for a in mydivs[1].findAll("div")[1:]]
        team = [a.text for a in mydivs[2].findAll("div")[1:]]
    except:
        return False
    for i in range(len(floor)):
        output[name[i]] = {'ceil':ceil[i], 'floor': floor[i], 'points':points[i], 'salary':salary[i], 'team':team[i]}
        #print(name[i], ceil[i], floor[i], points[i])
    return output
def scrape_data(date):
    try:
        get_base_page(date,Fanduel=False)
    except:
        return False
    time.sleep(8)
    try:
        actual_points = get_actual_points()
    except:
        return False
    contests = get_contests(date)
    projection = projections(date)
    return {"ACTUAL_POINTS": actual_points, "CONTESTS": contests, "PROJECTIONS": projection}
def scrape_all_data(Sdate = date(2017,9,1),Edate = date.today()):
    huge_data = {}
    sdate = Sdate   
    edate = Edate
    #driver = webdriver.Chrome('/usr/local/bin/chromedriver')

    delta = edate - sdate       

    for i in range(delta.days + 1):
        day = sdate + timedelta(days=i)
        
        text = day.strftime("%Y-%m-%d")
        print(text)
        #huge_data[text] = scrape_data(text)
        result = scrape_data(text)
        if result != False:
            huge_data[text] = result
    return huge_data

In [ ]:
result1 = scrape_all_data(date(2019,12,17), date(2019,12,19))

2019-12-17
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-12-17/sport/nba/slate/5df96d00e0c1a85c64852e2d/contest/5df96d020c6f395c5831e9da
{'Name': 'NBA $400K Big Jam [$100K to 1st]', 'Link': 'Contest', 'Prize Pool': '$400,000', 'Buy In': '$10', 'Top Prize': '$100,000', 'Max Entries': 150, 'Entries': 47059, 'Cash Line': 280.0, 'Winner': 'SEP6877', 'Winning Score': 334.0}
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-12-17/sport/nba/slate/5df96d00e0c1a85c64852e2d/contest/5df96d02e0c1a85c64852fe6
{'Name': 'NBA $100K Four Point Play [20 Entry Max]', 'Link': 'Contest', 'Prize Pool': '$100,000', 'Buy In': '$4', 'Top Prize': '$10,000', 'Max Entries': 20, 'Entries': 29727, 'Cash Line': 276.25, 'Winner': 'icealford', 'Winning Score': 347.0}
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-12-17/sport/nba/slate/5df96d00e0c1a85c64852e2d/contest/5dfa4cd940264e76d94a82c5
{'Name': 'NBA $40K Step-Back Fadeaway [$10K to 1st]', 'Link': 'Contest', 'Prize Pool': '

In [52]:
import pickle

filehandler = open("2018_season_second_half.pkl","wb")
pickle.dump(result1,filehandler)
filehandler.close()

In [53]:
projections("2019-12-01")

{'Tyus Jones': {'ceil': 47.41,
  'floor': 11.96,
  'points': 29.52,
  'salary': 3100.0,
  'team': 'MEM'},
 'Isaac Bonga': {'ceil': 27.86,
  'floor': 3.05,
  'points': 14.05,
  'salary': 1500.0,
  'team': 'WAS'},
 'Jalen Brunson': {'ceil': 23.05,
  'floor': 2.82,
  'points': 11.53,
  'salary': 1500.0,
  'team': 'DAL'},
 'Brandon Clarke': {'ceil': 65.14,
  'floor': 10.46,
  'points': 35.23,
  'salary': 4800.0,
  'team': 'MEM'},
 'Dillon Brooks': {'ceil': 54.47,
  'floor': 13.03,
  'points': 29.54,
  'salary': 4500.0,
  'team': 'MEM'},
 'Kevin Knox': {'ceil': 38.74,
  'floor': 6.22,
  'points': 21.46,
  'salary': 3300.0,
  'team': 'NYK'},
 'Jaren Jackson': {'ceil': 62.32,
  'floor': 13.3,
  'points': 35.47,
  'salary': 5600.0,
  'team': 'MEM'},
 'Solomon Hill': {'ceil': 37.58,
  'floor': 4.81,
  'points': 18.8,
  'salary': 3000.0,
  'team': 'MEM'},
 'DeAndre Jordan': {'ceil': 49.03,
  'floor': 10.01,
  'points': 29.27,
  'salary': 4700.0,
  'team': 'BKN'},
 'Jae Crowder': {'ceil': 50.59,
